# 📦 Online Retail Sales Data Analysis (with Cancellations)

#  1. Importing Libraries 

In [8]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [9]:
# Display settings

pd.set_option("display.float_format", lambda x: "%.2f" % x)
sns.set(style="whitegrid")

#  2. Importing Data

In [15]:
try:
    retail = pd.read_csv("/Users/micic/Downloads/data.csv", encoding='utf-8')
except UnicodeDecodeError:
    # Fallback to ISO-8859-1
    retail = pd.read_csv("/Users/micic/Downloads/data.csv", encoding='ISO-8859-1')

# SQL setup
pysqldf = lambda q: sqldf(q, globals())

# Example query
query = "SELECT * FROM retail LIMIT 5;"
print(pysqldf(query))

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55    17850.00  United Kingdom  
1  12/1/2010 8:26       3.39    17850.00  United Kingdom  
2  12/1/2010 8:26       2.75    17850.00  United Kingdom  
3  12/1/2010 8:26       3.39    17850.00  United Kingdom  
4  12/1/2010 8:26       3.39    17850.00  United Kingdom  


# 3. Data Cleaning 

In [16]:
# Data Description 

rows, columns = df.shape
print(f"Total Rows: {rows}")
print(f"Total Columns: {columns}\n")
print(df.describe())

# Missing/Null Values 

print(df.isnull().sum())


Total Rows: 406829
Total Columns: 11

       Quantity                    InvoiceDate  UnitPrice  CustomerID  \
count 406829.00                         406829  406829.00   406829.00   
mean      12.06  2011-07-10 16:30:57.879207424       3.46    15287.69   
min   -80995.00            2010-12-01 08:26:00       0.00    12346.00   
25%        2.00            2011-04-06 15:02:00       1.25    13953.00   
50%        5.00            2011-07-31 11:48:00       1.95    15152.00   
75%       12.00            2011-10-20 13:06:00       3.75    16791.00   
max    80995.00            2011-12-09 12:50:00   38970.00    18287.00   
std      248.69                            NaN      69.32     1713.60   

       TotalPrice  
count   406829.00  
mean        20.40  
min    -168469.60  
25%          4.20  
50%         11.10  
75%         19.50  
max     168469.60  
std        427.59  
InvoiceNo         0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
UnitPrice         0
Cust

In [53]:
df.dropna(subset=["CustomerID"], inplace=True)
df = df[df["Quantity"] != 0]   # keep cancellations but remove zero-quantity rows

df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Month"] = df["InvoiceDate"].dt.to_period("M")

# Flag cancellations
df["IsCancellation"] = df["InvoiceNo"].astype(str).str.startswith("C")

print(f"✅ Dataset shape after cleaning: {df.shape}")
print(f"✅ Total Cancellations: {df['IsCancellation'].sum()}")

✅ Dataset shape after cleaning: (406829, 11)
✅ Total Cancellations: 8905


# 3. Metrics & Analysis

## Sales & Revenue Metrics

### 1. Top Products by Revenue

In [18]:
query = """
SELECT Description, SUM(Quantity * UnitPrice) AS Revenue
FROM retail
WHERE InvoiceNo NOT LIKE 'C%'
GROUP BY Description
ORDER BY Revenue DESC
LIMIT 10;
"""
print(pysqldf(query))

                          Description   Revenue
0                      DOTCOM POSTAGE 206248.77
1            REGENCY CAKESTAND 3 TIER 174484.74
2         PAPER CRAFT , LITTLE BIRDIE 168469.60
3  WHITE HANGING HEART T-LIGHT HOLDER 106292.77
4                       PARTY BUNTING  99504.33
5             JUMBO BAG RED RETROSPOT  94340.05
6      MEDIUM CERAMIC TOP STORAGE JAR  81700.92
7                              Manual  78112.82
8                             POSTAGE  78101.88
9                  RABBIT NIGHT LIGHT  66964.99


### 2. Revenue by Country


In [20]:
query = """
SELECT Country, SUM(Quantity * UnitPrice) AS Revenue
FROM retail
WHERE InvoiceNo NOT LIKE 'C%'
GROUP BY Country
ORDER BY Revenue DESC
LIMIT 10;
"""
print(pysqldf(query))

          Country    Revenue
0  United Kingdom 9003097.96
1     Netherlands  285446.34
2            EIRE  283453.96
3         Germany  228867.14
4          France  209715.11
5       Australia  138521.31
6           Spain   61577.11
7     Switzerland   57089.90
8         Belgium   41196.34
9          Sweden   38378.33


### 3. Monthly Revenue Trend

In [25]:
query = """
SELECT strftime('%Y-%m', InvoiceDate) AS Month, SUM(Quantity * UnitPrice) AS Revenue
FROM retail
WHERE Quantity>0
GROUP BY strftime('%Y-%m', InvoiceDate)
ORDER BY Month;
"""
print(pysqldf(query))

  Month     Revenue
0  None 10644560.42


### 4. Average Order Value (AOV)

In [32]:
query = """
SELECT ROUND(SUM(Quantity * UnitPrice) * 1.0 / COUNT(DISTINCT InvoiceNo), 2) AS AOV
FROM retail  
"""
print(pysqldf(query))

     AOV
0 376.36


### 5. Customer Lifetime Value (CLV)

In [34]:
query = """
SELECT ROUND(AVG(CustomerRevenue), 2) AS CLV
FROM (
    SELECT CustomerID, SUM(Quantity * UnitPrice) AS CustomerRevenue
    FROM retail
    WHERE InvoiceNo NOT LIKE 'C%' AND Quantity > 0
    AND CustomerID is not NUll 
    GROUP BY CustomerID
);
"""
print(pysqldf(query))

      CLV
0 2053.79


### 6. Revenue per Customer by Country

In [39]:
query = """
WITH D1 AS (SELECT Country, CustomerID, ROUND(SUM(Quantity * UnitPrice), 2) AS Revenue
FROM retail
WHERE InvoiceNo NOT LIKE 'C%' AND Quantity > 0 and CustomerID is not NULL
GROUP BY Country, CustomerID) 

SELECT Country, AVG(Revenue) AS Revenue  FROM D1 GROUP BY Country
ORDER BY Revenue DESC
LIMIT 15 
"""
print(pysqldf(query))

            Country  Revenue
0              EIRE 70863.49
1       Netherlands 31716.26
2         Singapore 21279.29
3         Hong Kong 15691.80
4         Australia 15391.26
5            Sweden  4797.29
6             Japan  4677.05
7           Iceland  4310.00
8            Norway  3616.54
9       Switzerland  2595.00
10          Germany  2434.76
11           France  2383.13
12   United Kingdom  2295.54
13  Channel Islands  2272.27
14          Denmark  2106.15


## Customer Behavior Metrics


### 1. Repeat Purchase Rate

In [42]:
query = """
SELECT ROUND(COUNT(DISTINCT CASE WHEN InvoiceCount > 1 THEN CustomerID END) * 1.0 / COUNT(DISTINCT CustomerID), 2) AS RepeatPurchaseRate
FROM (
    SELECT CustomerID, COUNT(DISTINCT InvoiceNo) AS InvoiceCount
    FROM retail
    WHERE InvoiceNo NOT LIKE 'C%' AND Quantity > 0 AND CustomerID is not NULL
    GROUP BY CustomerID
);
"""
print(pysqldf(query))

   RepeatPurchaseRate
0                0.66


## Cancellation Metrics

### 1. Cancellation Summary

In [46]:
query = """
SELECT 
    COUNT(DISTINCT CASE WHEN InvoiceNo LIKE 'C%' THEN InvoiceNo END) AS Total_Cancellations,
    SUM(CASE WHEN InvoiceNo LIKE 'C%' THEN Quantity ELSE 0 END) AS Total_Cancelled_Items,
    ROUND(SUM(CASE WHEN InvoiceNo LIKE 'C%' THEN Quantity * UnitPrice ELSE 0 END), 2) AS Total_Cancelled_Value,
    ROUND(CAST(COUNT(DISTINCT CASE WHEN InvoiceNo LIKE 'C%' THEN InvoiceNo END) AS FLOAT)*100.00 / COUNT(DISTINCT InvoiceNo), 2) AS Cancellation_Invoices_percentage,
    ROUND(ABS(SUM(CASE WHEN InvoiceNo LIKE 'C%' THEN Quantity * UnitPrice ELSE 0 END)) * 100.0 / SUM(Quantity * UnitPrice), 2) AS Cancellation_Revenue_percentage
FROM retail;
"""
print(pysqldf(query))

   Total_Cancellations  Total_Cancelled_Items  Total_Cancelled_Value  \
0                 3836                -277574             -896812.49   

   Cancellation_Invoices_percentage  Cancellation_Revenue_percentage  
0                             14.81                             9.20  
